In [1]:
import nltk.tokenize
import string
import itertools
import unicodedata
from collections import defaultdict, namedtuple
import re
import numpy as np
import scipy.spatial.distance
import sys
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.decomposition import PCA
from collections import OrderedDict
import time

In [2]:
sys.path.append("wiki_search_engine")
import utils

In [14]:
def read_words_embeddings():
    with open("article_finder/data/wiki_embeddings_300.txt") as file_stream:
        lines = [x.strip() for x in file_stream.readlines()]
        return {x.split()[0].lower(): np.fromstring(" ".join(x.split()[1:]), dtype=float, sep=" ") for x in lines[1:]}
    

def get_wiki_words_embeddings(wiki_articles, words_base_forms, words_embeddings):
    wiki_words_embeddings = {}
    start_time = time.time()
    for wiki_article in wiki_articles:
        base_forms = set(itertools.chain(*utils.get_base_forms_from_article(words_base_forms, wiki_article)))
        for base_form in base_forms:
            if base_form not in words_embeddings or base_form in wiki_words_embeddings:
                continue
            word_embedding = words_embeddings[base_form]
            wiki_words_embeddings[base_form] = word_embedding / np.linalg.norm(word_embedding)
    return wiki_words_embeddings


def find_best_clusters_assignments(wiki_words_embeddings, clusters_count=100, **kwargs):
    ordered_wiki_words_embeddings = OrderedDict(wiki_words_embeddings)
    stacked_embeddings = np.array(list(ordered_wiki_words_embeddings.values()))
    clusters_creator = KMeans(n_clusters=clusters_count, **kwargs)
    predictions = clusters_creator.fit_predict(stacked_embeddings)
    terms_clusters = {}
    for index, word in enumerate(ordered_wiki_words_embeddings.keys()):
        terms_clusters[word] = predictions[index]
    return terms_clusters

    
def save_clusters_assignments(terms_clusters, name, folder="wiki_search_engine/data"):
    with open(f"{folder}/{name}.txt", "w") as file_stream:
        for term, cluster in terms_clusters.items():
            file_stream.write(f"{term} {cluster}\n")
        file_stream.flush()

In [4]:
wiki_articles = utils.read_wiki_articles(path="wiki_search_engine/data/wiki_slice.txt")
words_base_forms = utils.read_words_base_forms(path="wiki_search_engine/data/base_forms.txt")
words_embeddings = read_words_embeddings()

In [5]:
wiki_words_embeddings = get_wiki_words_embeddings(wiki_articles, words_base_forms, words_embeddings)
len(wiki_words_embeddings)

409983

In [ ]:
terms_clusters_100 = find_best_clusters_assignments(wiki_words_embeddings, clusters_count=100, max_iter=10, n_init=5)
save_clusters_assignments(terms_clusters_100, name="terms_clusters_100")

In [ ]:
terms_clusters_1000 = find_best_clusters_assignments(wiki_words_embeddings, clusters_count=1000, max_iter=10, n_init=5)
save_clusters_assignments(terms_clusters_1000, name="terms_clusters_1000")

In [18]:
cluster_479_terms = {x for x in terms_clusters_1000.keys() if terms_clusters_1000[x] == 479}

In [19]:
cluster_479_terms

{'aktualizować',
 'aktywować',
 'aplikować',
 'archiwizować',
 'bieżąco',
 'certyfikować',
 'cytować',
 'dobierać',
 'dobrać',
 'dodawać',
 'dodać',
 'dograć',
 'dokupić',
 'dokupować',
 'dopasowywać',
 'dopisać',
 'dopisywać',
 'dopracować',
 'dopracowywać',
 'dorzucić',
 'dostosować',
 'dostosowywać',
 'dowolnie',
 'doładować',
 'dołożyć',
 'drukować',
 'dystrybuować',
 'ebayu',
 'edytować',
 'eksportować',
 'ewentualnie',
 'filtrować',
 'formatować',
 'generować',
 'grupować',
 'implementować',
 'importować',
 'instalować',
 'integrować',
 'kasować',
 'katalogować',
 'kategoryzować',
 'klikać',
 'kliknąć',
 'kolorować',
 'komentować',
 'konstruować',
 'konwertować',
 'kopiować',
 'kwalifikować',
 'licytować',
 'logować',
 'lokalizować',
 'losować',
 'miksować',
 'modernizować',
 'modyfikować',
 'montować',
 'nadpisać',
 'nadsyłać',
 'nakierować',
 'naliczyć',
 'nanieść',
 'nanosić',
 'naprawiać',
 'naraz',
 'naświetlać',
 'numerować',
 'obsłużyć',
 'odblokowywać',
 'odejmować',
 'od

## Hierarchical clustering

In [12]:
top_terms_identifiers = utils.get_terms_identifiers(
    use_terms_clusters=True, path="wiki_search_engine/data/terms_clusters_100.txt"
)

In [19]:
hierarchical_terms_clusters = {}
for i in range(100):
    cluster_terms_embeddings = {
        x: words_embeddings[x]
        for x in top_terms_identifiers.keys() if top_terms_identifiers[x] == str(i)
    }    
    child_terms_clusters = find_best_clusters_assignments(
        cluster_terms_embeddings, clusters_count=100, max_iter=50, n_init=10, verbose=0
    )
    for term, child_cluster in child_terms_clusters.items():
        hierarchical_terms_clusters[term] = i * 100 + child_cluster        
save_clusters_assignments(hierarchical_terms_clusters, name="terms_clusters_10000")

In [21]:
[x for x in hierarchical_terms_clusters.keys() if hierarchical_terms_clusters[x] == 3909]

['programować',
 'kodować',
 'uruchamiać',
 'zaimplementować',
 'adresować',
 'skompilować',
 'kompilować',
 'zaadresować',
 'optymalizować',
 'zaszyfrować',
 'emulować',
 'skonfigurować',
 'implementować',
 'zaktualizować',
 'nadpisać',
 'konfigurować',
 'podpiąć',
 'wczytać',
 'przekonwertować',
 'zaimportować']

In [23]:
[x for x in hierarchical_terms_clusters.keys() if hierarchical_terms_clusters[x] == 9911]

['tworzyć',
 'pisać',
 'wydawać',
 'malować',
 'rzeźbić',
 'wychodzić',
 'czytać',
 'sporządzać',
 'układać',
 'drukować',
 'spisywać',
 'rysować',
 'kolekcjonować',
 'fotografować']